# <img src ='https://airsblobstorage.blob.core.windows.net/airstream/Asset 275.png' width="50px"> Access Azure Blob Storage from Azure Databricks using Azure Key Vault 

[link to tutorial](https://docs.microsoft.com/en-us/azure/databricks/scenarios/store-secrets-azure-key-vault)

Run the following command to mount your blob storage container. Remember to change the values for the following properties:

* your-container-name
* your-storage-account-name
* mount-name
* config-key
* scope-name
* key-name

In [0]:
#list all secret scopes 
dbutils.secrets.listScopes()

Out[3]: [SecretScope(name='akv-bck-scope'),
 SecretScope(name='akv-bck-scope-dataengineering'),
 SecretScope(name='akv-bck-scope-mnt-A'),
 SecretScope(name='akv-bck-scope-mnt-B'),
 SecretScope(name='andres-kvault-scope2'),
 SecretScope(name='databricks-bck-scope'),
 SecretScope(name='dataeng-akv-bck-scope'),
 SecretScope(name='finance-akv-bck-scope'),
 SecretScope(name='jdbc')]

In [0]:
dbutils.secrets.list("akv-bck-scope")

Out[8]: [SecretMetadata(key='ADLS--AccountKey'),
 SecretMetadata(key='ADLS--Cnx'),
 SecretMetadata(key='ADLSCnx--airstreampe'),
 SecretMetadata(key='ADLSCnx--peadls'),
 SecretMetadata(key='ADLSCnx-privateendpointandres'),
 SecretMetadata(key='AirsServicePrincipal'),
 SecretMetadata(key='ApplicationInsights--Cnx'),
 SecretMetadata(key='ApplicationInsights--ConnectionString'),
 SecretMetadata(key='Azure--SignalR--ConnectionString'),
 SecretMetadata(key='CloudStorage--AccountKey'),
 SecretMetadata(key='CloudStorage--Cnx'),
 SecretMetadata(key='ConnectionStrings--CnxDB'),
 SecretMetadata(key='ConnectionStrings--RedisCnx'),
 SecretMetadata(key='CosmosDB--AuthKey'),
 SecretMetadata(key='CosmosDB-Cnx'),
 SecretMetadata(key='Databricks--Token'),
 SecretMetadata(key='EventHubCnx-AirsHub'),
 SecretMetadata(key='EventHubCnx-Azurestream'),
 SecretMetadata(key='EventHubCnx-Azustream'),
 SecretMetadata(key='Gen2AccountKey'),
 SecretMetadata(key='Iot--IotHub'),
 SecretMetadata(key='Iot--StorageAccount'),
 SecretMetadata(key='SendGrid--SendGridKey'),
 SecretMetadata(key='SendGrid--SendGridPassword'),
 SecretMetadata(key='SendGrid--SendGridUser')]

In [0]:
dbutils.secrets.get(scope="akv-bck-scope", key="ApplicationInsights--ConnectionString")

Out[10]: '[REDACTED]'

In [0]:
dbutils.secrets.list("databricks-bck-scope")

Out[2]: [SecretMetadata(key='password'),
 SecretMetadata(key='ServicePrincipal'),
 SecretMetadata(key='ServicePrincipalPassword'),
 SecretMetadata(key='username')]

In [0]:
%scala

val driverClass = "com.microsoft.sqlserver.jdbc.SQLServerDriver"
val connectionProperties = new java.util.Properties()
connectionProperties.setProperty("Driver", driverClass)

val jdbcUsername = dbutils.secrets.get(scope = "databricks-bck-scope", key = "username")
val jdbcPassword = dbutils.secrets.get(scope = "databricks-bck-scope", key = "password")
val ServicePrincipal = dbutils.secrets.get(scope = "databricks-bck-scope", key = "ServicePrincipal")
val ServicePrincipalPassword = dbutils.secrets.get(scope = "databricks-bck-scope", key = "ServicePrincipalPassword")

driverClass: String = com.microsoft.sqlserver.jdbc.SQLServerDriver
connectionProperties: java.util.Properties = {Driver=com.microsoft.sqlserver.jdbc.SQLServerDriver}
jdbcUsername: String = [REDACTED]
jdbcPassword: String = [REDACTED]
ServicePrincipal: String = [REDACTED]
ServicePrincipalPassword: String = [REDACTED]

In [0]:
value = dbutils.secrets.get(scope="databricks-bck-scope", key="ServicePrincipalPassword")

for char in value:
    print(char, end=" ")

C . 2 7 Q ~ c x I Y 9 4 I K s E M J l X m p G Q g j M g _ H f N x x x x x x

In [0]:
dbutils.fs.mount(
source = "wasbs://apollo@airsblobstorage.blob.core.windows.net",
mount_point = "/mnt/apollo",
extra_configs = {"fs.azure.account.key.airsblobstorage.blob.core.windows.net":dbutils.secrets.get(scope = "akv-bck-scope", key = "CloudStorage--AccountKey")})

Out[2]: True

Run the following command to read a file in your blob storage container to a dataframe. Change the values in the command to match your mount name and file name.

In [0]:
df = spark.read.text("/mnt/apollo/apollo11.json")

Use the following command to display the contents of your file.

In [0]:
df.show()

+--------------------+
 value|
+--------------------+
 {|
 "id": "Apollo...|
 "Description"...|
 "Mission type...|
 "Operator": "...|
 "COSPAR ID": ...|
 "SATCAT": [ "...|
 "Mission dura...|
 "Spacecraft p...|
 "Spacecra...|
 "Manufact...|
 "Launch m...|
 "Landing ...|
 },|
 "Crew": {|
 "Crew siz...|
 "Members"...|
 "Callsign...|
 "On surfa...|
 },|
+--------------------+
only showing top 20 rows

To unmount your blob storage, run the following command

In [0]:
dbutils.fs.unmount("/mnt/apollo")

/mnt/apollo has been unmounted.
Out[6]: True

Notice that once the mount has been unmounted, you can no longer read from your blob storage account